### Installation

We follow the instructions from https://spacy.io/docs/usage/

In [ ]:
!sudo -H python3 -m pip install -U spacy

In [ ]:
# Support for english
!sudo python3 -m spacy download en

In [ ]:
# Support for german
!sudo python3 -m spacy download de

### Starting with Spacy

We first import the library and create an `nlp` variable, instantiated for English (`'en'`).

In [1]:
import spacy

# Load the space library, instantiated for English
#note: the first time you run spaCy in a file it takes a little while to load up its modules
nlp = spacy.load('en') 

From https://nicschrading.com/project/Intro-to-NLP-with-spaCy/

In [ ]:
text = """There is an art, it says, or rather, a knack to flying. 
The knack lies in learning how to throw yourself at the ground and miss.
In the beginning the Universe was created. This has made a lot of people
very angry and been widely regarded as a bad move.
This Prof. Panos, Ph.D. costs $12,345.67"""

In [ ]:
# all you have to do to parse text is this:
doc = nlp(text)

In [ ]:
tokens = [token for token in doc]
tokens[:10]

In [ ]:
# Let's look at the tokens
# All you have to do is iterate through the doc
# Each token is an object with lots of different properties
# A property with an underscore at the end returns the string representation
# while a property without the underscore returns an index (int) into spaCy's vocabulary
# The probability estimate is based on counts from a 3 billion word corpus
for i, token in enumerate(doc):
    print("original:", token.orth, token.orth_)
    print("lowercased:", token.lower, token.lower_)
    print("lemma:", token.lemma, token.lemma_)
    print("shape:", token.shape, token.shape_)
    print("prefix:", token.prefix, token.prefix_)
    print("suffix:", token.suffix, token.suffix_)
    print("part of speech:", token.pos_)
    print("log probability:", token.prob)
    print("Brown cluster id:", token.cluster)
    print("----------------------------------------")
    if i > 5:
        break

#### Get some data

First let's get a few text files, so that we can run our examples.

In [ ]:
!mkdir data
!curl -L 'https://raw.githubusercontent.com/cytora/pycon-nlp-in-10-lines/master/data/article.txt' -o data/article.txt
!curl -L 'https://raw.githubusercontent.com/cytora/pycon-nlp-in-10-lines/master/data/pride_and_prejudice.txt' -o data/pride_and_prejudice.txt
!curl -L 'https://raw.githubusercontent.com/cytora/pycon-nlp-in-10-lines/master/data/rand-terrorism-dataset.txt'  -o data/rand-terrorism-dataset.txt

Now we will read the text file and then we will use the `nlp` object from spacy to analyze the text.

In [ ]:
filename = "data/article.txt"
text = open(filename, 'r').read()
doc = nlp(text)

##### Print tokens

In [ ]:
# Print tokens, one token per line
# The enumerate function is just used to add a counter
for token in doc:
    print(token)

###### Print Sentences

In [ ]:
# Print the first 5 sentences (one sentence per line)
# The enumerate function is just used to add a counter
for i, sent in enumerate(doc.sents):
    print(i, "==>", sent)
    if i>5:
        break

#### Named Entities 

In [ ]:
entities = set([ent.lemma_ for ent in doc.ents])
# entities

In [2]:
text = '''
Abner of Burgos
First published Mon Jul 9, 2012; substantive revision Wed Aug 31, 2016
Abner of Burgos (Alfonso de Valladolid; c. 1260–1347) was perhaps the most important philosopher in the stream of Jewish Spanish rabbi-apostates in the 14th and 15th centuries. In the first part of his life, Abner was an Aristotelian Jewish philosopher. However, at the age of 60 (if Pablo de Santa Maria is a reliable source) after years of hesitation, he became a Neo-Platonic Christian, making him one of the very few philosophers (much less philosopher-rabbis) to change his philosophical opinion (and religion) during the Middle Ages. The majorities of his works are primarily polemical, and try to convince his former fellow Jews to become Christian, as he did. However, in his polemical writings, Abner built an original philosophical critique of both traditional Judaism and the Aristotelian (and Maimonidean) philosophical interpretation of Judaism. Another part of his philosophical aim was a very radical interpretation of Christianity aimed at convincing his Jewish audience that his Christianity was superior to traditional and philosophical Judaism. Abner was one of the first philosophical and scientific critics of Aristotle, as well as one of the more radical Neo-Platonic thinkers of Christianity during the Middle Ages. His works had a very important impact on Jewish philosophy at the latter end of the Middle Ages, and some of his opinions influenced, through Rabbi Hasdai Crescas, general Western philosophy.

1. Life
2. Works
3. The Trinity and Incarnation
4. Determinism
5. Critics of the Aristotelian science
6. The influence of Abner
Bibliography
Works by Abner
Secondary sources
Academic Tools
Other Internet Resources
Related Entries
1. Life
There are not many sources on the life of Abner. The majority of the sources are autobiographical passages in his works (especially Mostrador de Justicia). Abner was born around 1260, probably in the Jewish community of Burgos, one of the major communities in Castile. While still Jewish, Abner worked as a book trader, a rabbi, and he may have been a physician as well. During this time he was the head of a yeshiva (Jewish academy) in Burgos, but we do not know if this yeshiva was a public institution or just a private group of students that met in his home. We know from a former pupil, Rabbi Isaac Pulgar, and from the sources that he utilizes in his works, that during the Jewish period of his life, Abner was a philosopher, being a part of the Maimonidean-Averroist trend of Jewish philosophy at the time.

In spite of the majority opinion of modern scholars,[1] Abner probably was not a Kabbalist before his conversion to Christianity. This mistake is based essentially on two sources: the fact that some of Abner’s arguments for the Trinity and Incarnation can be interpreted as deriving from Kabbalistic sources, and the existence of another Rabbi Abner (not a common name for Jews of his time), a Kabbalist whom Rabbi Isaac of Acre cited many times. In my opinion, Abner’s argument for the Trinity and Incarnation is not based on Kabbalistic sources, but stems from his Neo-Platonic and standard Christian scriptural background.[2] Additionally, Abner (the apostate) did not use many important Kabbalistic sources.[3] His opinions are not the same as the other Rabbi Abner’s (the one that Rabbi Isaac of Acre quotes), and we can see that the two authors do not share the same background, nor do they use the same sources, with Abner utilizing almost entirely sources taken from the Jewish philosophical and traditional rabbinic corpuses.[4] almost all of Rabbi Abner’s sources are taken from the commentary of Nahmanides to the Torah, a book that Abner the apostate almost never quoted.

The beginning of Abner’s doubts about Judaism began with the incident of a false messiah in Avila in 1295. According to Abner, the Jews of the community, which is in North Castile, went outside the city to welcome the messiah, but instead of the messiah they received a rainfall of crosses that stuck to their clothes. Some of the Jews who witnessed the miracle went to Abner for advice. This event instilled in Abner some doubts about the question of the reason for the ongoing exile of Israel and the truth of Judaism. After twenty-two years of feeling ambivalent (i.e., around 1317) Abner wrote about his first revelation in a dream.[5] In his dream he saw a man that told him to be “a teacher of righteousness” (mostrador de justicia, not coincidentally the name of Abner’s main book) and to bring his people to freedom from exile. This dream just accents the uncertainty of the Jewish Rabbi. After three more years of studies he had a second dream wherein he had the same revelation, this time noticing that on the clothes the man wore were crosses like the cross of Jesus the Christian. This dream prompted him toward his final conversion around 1320/1321. After his conversion, Abner became the sacristan of the collegiate church of Valladolid and devoted himself to the propagation of Christianity amongst his former fellows Jews (he had come to the conclusion that conversion to Christianity was the only way to be saved from the Exile). We know he wrote an extended polemicist oeuvre and that he took part in the anti-Jewish polemic effort in North Castile around 1336–1339. There was a public disputation between Abner and a rabbi that occurred probably around 1335–1336 on the subject of the prayer of the Jews against the apostates. This dispute was the main reason for the decree against this part of the Jewish prayer (birkatha-minim) by Alfonso XI in 1336. Abner probably died in 1347 or later.

2. Works
Abner’s works may be divided into three different categories: polemic, scientific and philosophic.

Polemic works: Mostradorde Justicia is the earliest and most important work of those writings of Abner that have survived. Only a translation (probably by Abner himself) in Castilian exists today, whereas the original Hebrew was lost. The book starts with a brief introduction wherein Abner writes about the reasons for his conversion (the incident in Avila and his two dreams). The main part of the book is a debate between a Christian (the master) and a Jew (the rebel). In contrast to other works similar to this type of polemic debate, in the Mostrador, the rebel does not become a Christian, and despite his losing the debate, the Jew continues to believe in his ostensibly false religion. The book is divided into ten chapters, with each chapter being subdivided into a number of paragraphs. Each paragraph is a speech by one of the protagonists of the debate.[6]

Summary of the chapters:

I.
Abner indicated which books could be used as a basis for proof during polemic debates. In this chapter the master and the rebel agree that both sides may bring in proofs from philosophical arguments.
II.
Abner’s proof of the giving of a new law upon the coming of the Messiah.
III.
An explanation and justification for the revelation of a new law.
IV.
The arguments of those who deny that the messiah was to come and die and bring atonement for the sin of Adam; and what refutes those arguments.
V.
The arguments of the rebel which contradict the master’s position on the Trinity and the Incarnation of the person of the Son in the human form of the Christ, and the masters answer to those arguments.
VI.
The rebel’s arguments against incarnation of the divinity in the human form of the Christ and how the master tries to refute those arguments.
VII.
An attempt to prove that the hope that the Jews have in the coming of the prophesied Christ is a false hope, and that Jesus of Nazareth was the Christ who came according to the time that was indicated for the coming of the Christ in the books of the prophets and the sages.
VIII.
Arguments of the Jews that the Christ has not yet come, which is the basis for their claim that the Christians are the people of Edom or Esau, which was to be destroyed and to fall before the coming of the Christ; and an attempt to refute those arguments.
IX.
An attempt to prove that the hope that the Jews have for the coming of the Christ at any time is a false hope, and for this reason the Jews are still in exile from the Land of Israel.
X.
Abner’s attempt to prove that the Christians, according to the customs and commandments which they have in their laws, should be called the Holy Nation of Israel, and that the Jews, according to the customs and commandments which they have in the laws of the Talmud are not fit to return to the Land of Israel; and the rebel’s attempt to refute those arguments.
Responses to the Blasphemer (Teshuvot la-Meharef): this book survived both in the original Hebrew[7] and in a Castilian translation.[8] This book is a response to the different letters that Abner’s only known student, Rabbi Isaac Pulgar, wrote against his determinist and Christian writings.[9] Abner did not divide this book into formal sections, though the book contains four main subjects presented in a contiguous fashion: (1) an introduction wherein Abner attacks his former pupil on his philosophical position and other personal matters between the two; (2) the Trinity and how it can be based on philosophy, scripture and quotes from the Jewish rabbinic literature; (3) the necessary existence of the incarnation of divinity in the world; and (4) a proof of the aforementioned Christian beliefs derived from scripture and other Jewish sources (Talmud and Midrashim). In this last part, which takes up almost half the book, Abner argues that the messiah had already come and that Christianity is true.

While Abner’s arguments regarding the Trinity and Incarnation are generally more developed in Mostradorde Justicia, the philosophical importance of this work resides with his critique of the Jewish philosopher. Here Abner distinguishes between the regular Jewish people (that have some hope to become Christian and to be saved) and Jewish philosophers (like Pulgar). The latter are hopeless and are left with no religion, mostly due to their opinions that only the people who attain scientific knowledge have some kind of existence after death. Due to their feeling elevated over others, these philosophers have no hope of improving and accepting the truth of Christianity. Parts of Abner’s critique of Jewish philosophers in this book most probably influenced Crescas.

Libro de la Ley (Book of the Law). This work survived only fragmentally[10] in Castilian. The major argument of Abner in this book is that the Jewish people forgot the secrets of the Torah, which are manifested by the Christian doctrines, especially the Trinity.

Sefer Teshuvot ha-Meshubot (The Book of Responses to the Apostasies) exists only in Hebrew.[11] There are three polemic letters[12] that deal especially with scriptural arguments on the subject of the coming of the messiah.[13]

3. The Trinity and Incarnation
The two major subjects of the philosophical polemical works of Abner are the Trinity and Incarnation. Abner had an original view on the Trinity. In his opinion, one can prove philosophically the reality of the Trinity.[14] He claimed that only the division of the divine source of the world can explain the diversity of the world. The infinite force of God (the father) would burn the finite matter of the world if it were not for some “transformer” that adapted the divine force to the finitude of matter (the son). Abner distinguished between two parts of the son. The superior son is part of the transcendent divinity. The inferior son is the divine essence in all the different parts of the world. The transformer of the divine force and essence is the superior son.

Regarding the theory of the divine attributes, Abner also had an original opinion. He distinguished between the attributes that are the essence of God and the attributes that are essential to God. The attributes of the essence of God divide themselves only within the personas of the Trinity. The rest of the attributes are only essential to God (and could be attributed to any one of the personas). This division of the divine attributes influenced Rabbi Hasdai Crescas,who later influenced Spinoza.

Abner’s opinion on Incarnation was also very original. According to Abner, the essence of God is in the entire world. The world is composed only of corrupted matter and of divine essence in different degrees of purity. Even the most corrupted matter has inside itself some divine essence. The divine essence gives the corporeal form to matter and produces in it its dimensions. The uniqueness of a human being is the capacity to purify one’s matter, thereby reaching a higher degree of divine essence. According to Abner, what made Jesus unique was that he was born of the highest matter, enabling him to unite with the highest degree of divine essence that a human can attain. Abner thought that the superior son does not incarnate in this world. Jesus was only the highest degree of the inferior son that is present in the essences of the entire world.

The opinion of Abner on Incarnation is closely related to his view of the doctrine of Original Sin. In Abner’s opinion, the reason for the original sin was Adam’s lack of comprehension. Adam thought that his intellect, which is an incarnation of divinity in people, was God, and he therefore wrongly concluded that he was God. In order to fix Adam’s error, humanity needed the Torah of Moses, which emphasized the unity of God in an exaggerated form and therefore explained (somewhat inaccurately) that God does not incarnate in the World at all. Only after this critical step, humanity was able to understand that, though there is divinity in humans, this divinity is not an independent God, but rather a part of the divine essence in the whole world. The Law of Moses that came to purify the world from idolatry has some problems. It does not enable the full emancipation of humanity from sin and error. Abner claimed that the negation of the incarnation of God in the world leads to negating the possibility of life after death. The possibility of life after death comes from understanding the incarnation of divinity in all humans. Negating life after death causes immorality. The Torah has to put up with all these inaccuracies in order to achieve its main goal, which is taking humanity out of the sin of idolatry (by believing that God is one). The outcome, though better than its predecessor, is ultimately a lost situation which humanity cannot overcome on its own. For that reason, God sent Jesus (who was born with a higher degree of matter and divine essence). Due to the divinity within Jesus, his miracles and his resurrection, people are able to understand that within everything in the world there is some divine essence that is part of the inferior son, and that the origin of all the divinity in the world is the superior son. This understanding enables salvation from the original sin and represents a true understanding of the relation between God and the world. The role of the sacramentental Mass is to remind us of the divine aspect of Jesus’ essence, as well as of all the miracles that Jesus performed and to continue their influence in our present time.

We see that this opinion regarding the Trinity and Incarnation contradict the official dogma of the Catholic Church at the time of Abner. In his opinion, the incarnation of divinity in Jesus did not bypass the regular process of the emanation of the divine essence in the world. Rather, the manifestation of Jesus represented the pinnacle of incarnation of divinity, which, though indeed present in the entire world, yet in Jesus occurred in the purest form possible. This opinion of Abner, similar to his other philosophical views, was a part of the radical Neo-Platonic interpretation of Christianity. The difference between him and the other Neo-Platonic Christian philosophers, like Master Eckard, is the different sources from which they were influenced. Abner did not utilize the traditional Neo-Platonic Christian sources, like Dionysius the Areopagite. We do not see in any part of his work that he was aware of the existence of these sources. On the other hand, Abner did utilize some Arabic and Jewish sources. The origin of his uncommon position on Christianity is found in his opposition to the common Aristotelian-Maimonidean-Averroist trend of Jewish philosophy. Abner, similar to Spinoza after him, thought that the explanations of these thinkers on the question of the creation of the world by a perfect God were not sufficient and ultimately inaccurate. In place of these responses, Abner proposed his radical interpretation of the Christian doctrine of the Trinity, an interpretation that was uninfluenced by the theological Christian tradition.

4. Determinism
One of the more important philosophical opinions of Abner was his deterministic view.[15] Abner devoted his book Ofrenda de Zelos (Minhat Kena’ot in the original lost Hebrew text), along with other works that have since been lost, to the question of determinism and free will.[16] In the beginning of this book, Abner describes the opinion of his former pupil Rabbi Isaac Pulgar, who denies God’s foreknowledge of human decisions. After the explication of this opinion and its negation, Abner explains his deterministic opinion. Abner believed that people have free will in a limited sense, but act in a determined way in the broader sense. People are free in their relation to themselves, meaning that if a person is separated from the causes that influence him or her (e.g., education, society’s influences) while debating between two options, one is then really free to choose either of the two options. In this specific case a person can then utilize his or her will and choose freely between the two options. However, if one takes into account all the causes that influence his or her will, we may conclude that these causes limits that person’s will to only one of the two options. In the opinion of Abner, people are like wax, as follows: Wax may be melted and sculpted into many different shapes. However, the person who sculpts the wax determines its present form as only one of those options. The same is true about people: In themselves, they have the ability to choose between different possibilities, but their relation to the outside world determines their choices. We can see from this example how strong the determinist view of Abner was. It is important to note that Abner defines the will of people as an accord between the attractive force and the imagination, two forces common to humans and animals.

Abner explains in the same way the meaning of possibility and accident. Things are only possible in their relation to themselves. In relation to the entire world and its causality, everything is truly determined. Accidents are only accidents relative to the seeming unlikelihood of their occurrence. However, relative to all the causal processes of the world, we see that even in the most unusual of circumstances the occurrence of what appears to be an accident in fact had to happen. In short, there are no accidents (according to the popular meaning of accident) for Abner.

This deterministic description of the world (based on rational arguments) resolves the theological problem of the contradiction between the free will of people and the foreknowledge of God. In the opinion of Abner free will does not really exist. For him, God is the first cause; He knows all the laws of the world and therefore knows everything that He has determined.

This deterministic description leads to two theological problems. The first is why God sent commandments to people who cannot truly decide whether to obey them or disobey them. The second question that arises is how people can receive reward or punishment for actions and decisions that they do not really have any control over? Abner answers the second question by explaining that forbidden things are naturally bad. The punishment of the sinner is not a special act of divine providence; rather it is the natural consequence of his bad action. This leads us to the answer to the first question. According to Abner, the goal of the Torah is to influence people to do what is right. The Torah and prophets (and God who sent them) act in the world in conformity to His nature. The only way to influence someone to do something is to give him enough causes to do it. The Torah is only another cause that influences people to do what is good. Abner compares reward and punishment to a father that obligates his son to take medication. The medicine will always work even if the son takes it unhappily. The same is true with regard to all actions of people. When people do what is right, they believe they will receive a reward, while if they do what is wrong, they believe they will receive a punishment—even if they do not essentially have the choice to do anything else.

It is interesting to note that this philosophical book that Abner wrote at the end of his life, more than twenty years after his conversion, is almost without Christian references.[17] In spite of this, the book is full of references to Jewish sources, including the Talmud and quotations of Jewish authorities like Maimonides.[18] We see that Abner, even many years after his conversion, wrote to the Jewish community and was part of the internal Jewish debate about the question of determinism. In fact, Abner is the first medieval Jewish philosopher to argue for a deterministic explanation of the world. His opinion influenced Rabbi Hasdai Crescas (who paraphrased an important part of Ofrenda de Zelos in his major philosophical work The Light of the Lord[19]), and through him, Spinoza. Some Jewish philosophers, like Rabbi Isaac Pulgar, Rabbi Mosses from Narbonne, and Rabbi Josef Ben Shem Tov[20] respond explicitly to the determinist opinion of Abner, such that there is no doubt that his opinion had a major influence on the internal Jewish debate on the question of free will throughout the 14th and 15th centuries.

5. Critics of the Aristotelian science
As we stated before, the major work of science of Abner (the New Philosophy) is lost. We can still try to reconstruct some of his scientific opinions through some parts of his other works, specifically the existing work Meyasher Aqob, Rabbi Isaac Pulgar’s brief summary of the lost work, and some scientific passages in Mostrador de Justicia.[21]

The first interesting opinion of Abner is his view on prime matter. Abner denied a necessary link, which the Aristotelians affirmed, between bodies and accidents (magnitude, weight, quantity, etc.), maintaining that a body untainted by all these accidents can nonetheless have actual existence. He explicated his view in the dogmatic context of the question of the rite of Mass.[22] The rebel’s main philosophical arguments against the Mass are as follows: (1) How can several bodies occupy the same place? (2) How can Jesus’ body be in the bread and wine without changing them? (3) How can the same body be two different materials at the same time? (4) How can one debase Jesus by eating and digesting him? At the start of his answer Abner emphasizes that the Mass is miraculous. Nevertheless, he does go on to offer a scientific explanation of transubstantiation. For Abner, everybody has a layer of pure divine existence, to which are superadded other forms and accidents; but if the body is stripped of all of these accretions it will be reduced to the original divine substance only. In this state the body will have no boundary, quantity, or weight to occupy the universe, and consequently is infinite. This body is the foundation of the entire material world, and normally—with the exception of the sacrament of the Mass and Incarnation—it does not exist without accidents (at least not in the sublunary world). This matter is identified with the flesh of Jesus, which has divine existence only, which means that Jesus’ body is identified with prime matter.

The above description eliminates for Abner all of the difficulties posed by the rebel. The problem of the co-presence of two bodies in one place is eliminated, because all bodies, once they have been stripped of their form and accidents, incorporate the infinite body of Jesus, which fills the entire universe. After these forms have been removed, the bread and wine are identified with Jesus’ body. The very same principle eliminates the problem of the simultaneous existence of Jesus’ body in several places, since his body fills the entire universe and transcends all definitions of place. Because his body also transcends the definition of quantity, the problem of limited quantity is removed as well.

Another subject about which Abner critiques Aristotelian science is the definition of place and the relation between place and body. In the thought of Aristotle, the definition of place is ‘the limit of the encompassing body’. In his opinion, place has an important role in movement. The definition of a natural movement is the movement of a body to its natural place in the world. Aristotle explains that both the natural place attracts the body and that the body moves on his own to its natural place.

Abner distinguishes between the physical object and extension or dimension. According to Abner, dimension is “a simple and subtle quantity detached from all movable bodies”; in other words, it is space, independent of any physical body. Dimension is measurable per se, and is not linked to what occupies it. Nevertheless, Abner’s definition of place is still associated with body: “place” is the dimension occupied by a body, and it is the body that turns it into “place.” According to Abner, the movement of bodies is not due to their being attracted to some place where they must rest before resuming their movement; rather, they move because of the interaction of several forces, quite independent of any particular place. A body that is thrown upward rises, but if it suddenly begins to fall because of an opposing force, there would not be a moment of rest between the two opposing motions.

Thus Abner dismisses Aristotle’s concept that a body is attracted to its natural place. Different places are not characterized by different qualities (inasmuch as all places are equal and are merely points in space). Hence a body has no cause to move to a particular place and no place is a final cause of its motion.

We know from Rabbi Isaac Pulgar that Abner also criticized the opinions of Aristotle on the possibility of a void and argues for the existence of voids in his lost scientific book The New Philosophy. Because of the influence of Abner on Rabbi Hasdai Crescas, we can assume that his criticism is closely related to the criticism of Crescas in the beginning of the first speech of Or Hashem

Abner wrote his criticism of Artistotelian science at the beginning of the 14th century, approximately the same time that Ockham wrote his criticism of this science. In the writing and the thought of Abner we do not see any kind of influence of Ockham or of other late scholastics (the only scholastic philosopher that he cites is Thomas Aquinas and there are no quotations from Christian sources on a scientific subject). It is interesting to note that the systematic criticism of Aristotelian science started approximately at the same time in the Jewish milieu (with the writings of Abner) and in the Christian milieu, with probably no relation one to the other.

Abner’s criticism of Aristotelian science had an important influence on Crescas’s thought, and through him to Pico Della Mirandola and general Western philosophy.

6. The influence of Abner
One must distinguish between two kinds of influence Abner had. Abner influenced with his polemical works the Jewish-Christian debate until the end of the Middle Ages. Some born-Christians, like Alfonso de Espina, and especially Jewish apostates, like Pablo de Santa Maria, utilized his polemical works. Against this trend, there are more than five Jewish polemicist works that explicitly answer the polemical arguments of Abner.[23] Despite the utilization of Abner in debates, his works had almost no philosophical influence on Christian theology because the Christian participants in the debates utilized the quotations of the Jewish sources by Abner and not his theological arguments as such, which in their content were very different from the official Catholic dogma.

The second way in which Abner had lasting influence was on the purely philosophical plane. It should be noted that the direct philosophical influence of Abner was only on the Jewish community. In fact we do not see that any Christian philosophers read the works of Jewish apostates in any other forum apart from the purely polemical one. By contrast, in the Jewish community the works of Abner had an important impact until the expulsion of the Jews from Spain in 1492. The deterministic opinion of Abner is with no doubt one of the major causes to the extensive number of works written on this subject in the 14th and 15th centuries by Jewish philosophers. For the most part, these works were against the opinion of Abner. The most impressive philosophical influence of Abner was on Rabbi Hasdai Crescas (died in 1410/1411). Crescas was the main Jewish philosopher during the end of the Middle Ages. Despite their opposite positions in the Jewish-Christian debate (Crescas wrote one of the more philosophical anti-Christian works),[24] Crescas borrowed some of the major original philosophical opinions of Abner, including determinism, the difference between the attributes of the essence and the attributes that are essential, and the criticism of Aristotelian science. With regard to this entire subject, Crescas molded his opinion using Abner combined with other sources. In spite of this important change, we can clearly see signs of the philosophy of Abner in the writing of Crescas, and these signs constitute nearly all the post medieval influence of Abner on Jewish and general Western philosophy.
'''

In [3]:
doc = nlp(text)

In [6]:
entities_with_type = set([ent.lemma_+" # "+ent.label_ for ent in doc.ents if ent.label_=='PERSON' ])
entities_with_type

{'abner \n bibliography # PERSON',
 'abner # PERSON',
 'abner ’s # PERSON',
 'adam # PERSON',
 'adam ’s # PERSON',
 'alfonso de espina # PERSON',
 'alfonso de valladolid # PERSON',
 'alfonso xi # PERSON',
 'aristotelian # PERSON',
 'aristotle # PERSON',
 'blasphemer # PERSON',
 'burgos # PERSON',
 'crescas # PERSON',
 'crescas ’s # PERSON',
 'god # PERSON',
 'hasdai crescas # PERSON',
 'i. \n abner # PERSON',
 'isaac pulgar ’s # PERSON',
 'iv # PERSON',
 'jesus # PERSON',
 'josef ben shem tov[20 # PERSON',
 'judaism # PERSON',
 'law # PERSON',
 'libro de la ley # PERSON',
 'maimonidean # PERSON',
 'mass # PERSON',
 'meyasher aqob # PERSON',
 'midrashim # PERSON',
 'moses # PERSON',
 'mostrador # PERSON',
 'mostradorde justicia # PERSON',
 'ockham # PERSON',
 'pablo de santa maria # PERSON',
 'pico della mirandola # PERSON',
 'rabbi abner ’s # PERSON',
 'rabbi hasdai crescas # PERSON',
 'rabbi isaac # PERSON',
 'rabbi isaac pulgar # PERSON',
 'rabbi mosses # PERSON',
 'sefer teshuvot # 

#### Noun chunks

In [ ]:
chunks = [chunk.lemma_ for chunk in doc.noun_chunks if chunk.lemma_ not in entities]
chunks

In [ ]:
from collections import Counter

keywords = Counter()
for chunk in chunks:
    # print(chunk, nlp.vocab[chunk].prob )
    if nlp.vocab[chunk].prob < -8: # probablity value -8 is arbitrarily selected threshold
        keywords[chunk] += 1

keywords.most_common(20)

In [ ]:
for ent1 in doc.ents:
    for ent2 in doc.ents:
        similarity = ent1.similarity(ent2)
        if similarity > 0.5:
            print('{} - {} - {}' .format(ent1, ent2, similarity))

In [ ]:
from numpy import dot
from numpy.linalg import norm

# cosine similarity
cosine = lambda v1, v2: dot(v1, v2) / (norm(v1) * norm(v2))

# Let's see if it can figure out this analogy
# B is to A as C is to ???
a = nlp.vocab['London']
b = nlp.vocab['UK']
c = nlp.vocab['France']

result = a.vector - b.vector + c.vector

# gather all known words, take only the lowercased versions
allWords = list({w for w in nlp.vocab if w.has_vector and w.is_title and w.lower_ not in set({a.lower_,b.lower_,c.lower_})})
# sort by similarity to the result
allWords.sort(key=lambda w: cosine(w.vector, result))
allWords.reverse()
print("\n----------------------------\nTop 3 closest results:")
for word in allWords[:3]:   
    print(word.orth_)
    
# it got it! 

In [ ]:
filename = "data/rand-terrorism-dataset.txt"
text = open(filename, 'r').read()
doc = nlp(text)

In [ ]:
filename = "data/pride_and_prejudice.txt"
text = open(filename, 'r').read()
doc = nlp(text)